In [ ]:
import os
import pandas as pd
import numpy as np

# Set the directory path
#directory = 

# Initialize an empty list to store the combined DataFrames
combined_dfs = []

for year in range(2007, 2025):
    file_path = os.path.join(directory, f"market_history_{year}.csv")
    
    df = pd.read_csv(file_path, usecols=["Day", "Market Participant", "Total Trade Count", "Total Shares"])
    
    nyse_dn = df[(df["Market Participant"] == "NYSE (DN)")]
    nyse_n = df[(df["Market Participant"] == "NYSE (N)")]
    nasdaq_t = df[df["Market Participant"].str.upper().isin(["NASDAQ (DQ)"])]
    nasdaq_q = df[df["Market Participant"].str.upper().isin(["NASDAQ (Q)"])]
    
    nyse_combined = nyse_dn.set_index("Day")[["Total Trade Count", "Total Shares"]].add(
        nyse_n.set_index("Day")[["Total Trade Count", "Total Shares"]], fill_value=0
    )
    nyse_combined["Market Participant"] = "NYSE"
    nyse_combined.reset_index(inplace=True)
    
    nasdaq_combined = nasdaq_t.set_index("Day")[["Total Trade Count", "Total Shares"]].add(
        nasdaq_q.set_index("Day")[["Total Trade Count", "Total Shares"]], fill_value=0
    )
    nasdaq_combined["Market Participant"] = "NASDAQ"
    nasdaq_combined.reset_index(inplace=True)
    
    combined_df = pd.concat([nyse_combined, nasdaq_combined])
    
    combined_df["Total Trade Count"] = np.log(combined_df["Total Trade Count"])
    combined_df["Total Shares"] = np.log(combined_df["Total Shares"])
    
    combined_dfs.append(combined_df)

consolidated_df = pd.concat(combined_dfs, ignore_index=True)

print(consolidated_df)